# **Construindo um Chatbot com a API Gemini, Fundamentado em Documentos de texto**

**Além do Conhecimento Geral para uma IA Fundamentada**
A tarefa de criar um chatbot que responda a perguntas de usuários é uma aplicação comum de Grandes Modelos de Linguagem (LLMs). No entanto, um desafio significativo com LLMs de propósito geral, como o Gemini, é sua tendência a "alucinar".

Para aplicações empresariais, acadêmicas ou especializadas, onde a precisão e a rastreabilidade são primordiais, as respostas do chatbot devem ser estritamente limitadas a um conjunto de documentos.

A solução para este desafio é uma arquitetura conhecida como **Geração Aumentada por Recuperação (RAG)**. Em vez de simplesmente fazer uma pergunta ao LLM e esperar que ele "saiba" a resposta com base em seu vasto treinamento prévio, o RAG introduz um passo intermediário crucial.

![](https://static.wixstatic.com/media/cfe500_0546c1c5b8b3430f90c039aaa4ab71e2~mv2.jpg/v1/fill/w_740,h_438,al_c,q_80,usm_0.66_1.00_0.01,enc_avif,quality_auto/cfe500_0546c1c5b8b3430f90c039aaa4ab71e2~mv2.jpg)

1. **Prompt do Usuário**: Primeiro, o usuário insere um prompt com a pergunta (query) no sistema.

2. **Busca por Informação**: A query é usada para consultar fontes de conhecimento (como documentos PDF, TXT, etc.) previamente indexadas.

3. **Recuperação**: O sistema retorna ***trechos relevantes dos documentos*** que servirão como ***contexto aumentado***.

4. **Preparação do Prompt**: O contexto recuperado é combinado com a query original, formando um novo prompt fundamentado.

5. **Geração da Resposta**: O novo prompt é enviado ao LLM (como o Gemini), que gera uma resposta com base exclusivamente nas informações recuperadas.

A arquitetura RAG pode ser definida como um pipeline de três estágios principais: **Indexação, Recuperação e Geração**.


## 1. **Indexação**

A Indexação na pratica é um processo em 3 etapas:

1.1 **Ingestão**: Carregar e analisar os arquivos brutos PDF e TXT do seu corpus de conhecimento.


Instalação das dependencias:
* google-generativeai: O SDK oficial do Google para interagir com a API Gemini.

In [2]:
!pip install -U -q google-generativeai


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Chave da API do Gemini

In [ ]:
GOOGLE_API_KEY = "Sua API Key aqui"

In [ ]:
# no google colab use:
from google.colab import userdata # em projetos reais use dotenv para "esconder" sua API KEY
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
# no vscode use:
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

 Importando texto de PDF

 Para este exemplo usaremos os arquivos PDF em `content`

Instalação das dependencias:
* pypdf: Biblioteca para extrair texto de arquivos PDF.

In [6]:
!pip install -U -q pypdf


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Ingestão de documentos (PDF/TXT) por diretório, com título por arquivo
import os
from pypdf import PdfReader

# Caminho para os documentos (use relativo para portabilidade)
dir_path = r"\content"

# Lista de documentos com título e texto
documents = []  # cada item: {"title": str, "text": str}

for fname in os.listdir(dir_path):
    fpath = os.path.join(dir_path, fname)
    if not os.path.isfile(fpath):
        continue
    ext = os.path.splitext(fname)[1].lower()
    title = os.path.splitext(fname)[0]  # nome do arquivo sem extensão
    if ext == ".pdf":
        reader = PdfReader(fpath)
        pages = reader.pages
        doc_text = ''.join((page.extract_text() or '') for page in pages)
    elif ext == ".txt":
        with open(fpath, "r", encoding="utf-8", errors="ignore") as f:
            doc_text = f.read()
    else:
        continue
    if doc_text.strip():
        documents.append({"title": title, "text": doc_text})

# Mantém a variável `text` por compatibilidade com células seguintes
text = "\n\n".join(doc["text"] for doc in documents)

1.2 **Divisão (Chunking)**: Dividir estrategicamente o texto dos documentos em pedaços menores e gerenciáveis, conhecidos como "chunks".

Esta etapa é fundamental por duas razões principais:

 - **Limites de Contexto do Modelo**: Os modelos de embedding e os LLMs têm um limite máximo de tokens que podem processar de uma só vez. Enviar um documento inteiro (100 páginas, por exemplo) excederia esse limite.

- **Precisão da Recuperação**: Se você incorporar um documento inteiro em um único vetor, o vetor representará o significado médio de todo o documento. Quando um usuário faz uma pergunta específica, é improvável que a média de todo o documento seja a correspondência mais próxima. Chunks menores e mais focados permitem uma recuperação muito mais precisa e relevante.

A escolha da estratégia de "chunking" é uma decisão de design importante. As estratégias comuns incluem:

- **Tamanho Fixo (Fixed-Size)**: A abordagem mais simples, dividindo o texto em chunks de N caracteres ou tokens. Sua principal desvantagem é que pode cortar frases ou parágrafos no meio, quebrando o contexto semântico.
- **Semântica (Semantic)**: Tenta dividir o texto em limites lógicos, como frases ou parágrafos.
- **Recursiva (Recursive)**: Uma abordagem mais sofisticada que tenta dividir o texto usando uma hierarquia de separadores. Por exemplo, primeiro tenta dividir por parágrafos (\n\n). Se os chunks resultantes ainda forem muito grandes, ele os divide por frases, e assim por diante.

Para a maioria dos documentos baseados em texto, a **Divisão Recursiva de Caracteres (Recursive Character Text Splitting)** oferece o melhor equilíbrio entre simplicidade e preservação do contexto semântico. Ela respeita a estrutura natural do documento, tentando manter os parágrafos e as frases intactos sempre que possível.

Dois parâmetros chave nesta estratégia são `chunk_size` e `chunk_overlap`. `chunk_size` define o tamanho máximo de cada chunk. `chunk_overlap` especifica quantos caracteres do final de um chunk devem ser repetidos no início do próximo.

![chunk overlap](https://cdn.analyticsvidhya.com/wp-content/uploads/2025/02/unnamed-1-67a0e0c9ca199-1.webp)

Instalação das dependencias:
* langchain-text-splitters: Uma ferramenta útil para implementar estratégias de divisão de texto.

In [58]:
!pip install -U -q langchain-text-splitters

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=60,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)   

Podemos estruturar os chunks adicionando metadados como, por exemplo, o título ou nome do documento.

In [18]:
chunks = []  # cada item: {"text": str, "title": str}
for doc in documents:
    for chunk_text in text_splitter.split_text(doc["text"]):
        chunks.append({"text": chunk_text, "title": doc["title"]})

In [ ]:
print(f"Total de chunks criados: {len(chunks)}")
print("Exemplo título do primeiro chunk:", chunks[0]["title"])

In [ ]:
chunks[0]

In [ ]:
chunks[1]

1.3 **Embedding e Indexação**: Converter cada "chunk" de texto em uma representação numérica (um vetor de embedding) e armazenar esses vetores para busca rápida.

 - *Embedding:*

In [23]:
models_names = []
for model in genai.list_models():
  if 'embedContent' in model.supported_generation_methods:
    print(model.name)
    models_names.append(model.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


In [24]:
models_names[1]

'models/text-embedding-004'

In [25]:
%%time
embedded_chunks = []
for chunk in chunks:
    try:
        result = genai.embed_content(
            model=models_names[1],
            content=chunk["text"],
            task_type="RETRIEVAL_DOCUMENT"
        )
        chunk["embedding"] = result['embedding']
        embedded_chunks.append(chunk)
    except Exception as e:
        print(f"Erro ao gerar embedding para o chunk: {chunk[:50]}... | Erro: {e}")

CPU times: total: 750 ms
Wall time: 2min 21s


In [27]:
print(embedded_chunks[0])

{'text': 'MINISTÉRIO DA JUSTIÇA\nSecretaria Nacional do Consumidor\nCÓDIGO DE PROTEÇÃO \nE DEFESA DO CONSUMIDOR\nNova edição revista, atualizada e ampliada com \nos  Decretos nº 7.962, de 15 de março de 2013 \ne  nº 7.963, de 15 de março de 2013\nBrasília, 2013\nBrasil\n[Código de Proteção e Defesa do Consumidor(1990)]\nCódigo de Defesa do Consumidor - Nova ed. rev., atu-\nal. e ampl. com os Decretos nº 2.181, de 20 de março de \n1997 e nº 7936, de 15 de março de 2013 - Brasília : Minis-\ntério da Justiça, 2013\n156 p. \n1. Proteção ao consumidor - legislação - Brasil.\nCDD 341.37Sumário\nAPRESENTAÇÃO .............................................................11\nLEI Nº 8.078, DE 11 DE SETEMBRO DE 1990. ...............13\nTÍTULO I - Dos Direitos do Consumidor............................ 15\nCAPÍTULO I - Disposições Gerais ..........................15\nCAPÍTULO II - Da Política Nacional de Relações de \nConsumo .................................................................16', 'tit

- *Indexação:*

Os embeddings precisam ser armazenados em um local que permita uma busca por similaridade extremamente rápida. Bancos de dados relacionais tradicionais não são projetados para este tipo de operação. Em vez disso, usamos **armazenamentos de vetores (vector stores)**. Seguem duas opções:
1. **[ChromaDB](https://www.trychroma.com/)** é um banco de dados vetorial de código aberto, construído especificamente para aplicações de IA. Ele abstrai grande parte da complexidade. Com uma API simples, ele gerencia o armazenamento de embeddings, documentos e metadados em um único local.  

2. **[FAISS (Facebook AI Similarity Search)](https://ai.meta.com/tools/faiss/)** não é um banco de dados, mas sim uma biblioteca de código aberto altamente otimizada para busca de similaridade em conjuntos densos de vetores.

Para os propósitos deste guia, que visa construir um protótipo funcional, **ChromaDB** é a escolha recomendada devido à sua simplicidade e ciclo de desenvolvimento rápido.

Instalação das dependencias:
* ChromaDB: Um banco de dados vetorial de código aberto projetado para IA.

In [28]:
!pip install -U -q chromadb


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import chromadb

In [33]:
DB_PATH = "./chroma_db"
COLLECTION_NAME = "document_chatbot"

In [34]:
# Inicializa o cliente persistente
client = chromadb.PersistentClient(path=DB_PATH)

# Cria a coleção (db)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# Prepara os data for addition
ids = [f"chunk_{i}" for i in range(len(embedded_chunks))]
embeddings = [chunk["embedding"] for chunk in embedded_chunks]
documents = [chunk["text"] for chunk in embedded_chunks]
metadatas = [{"title": chunk["title"]} for chunk in embedded_chunks]

Adiciona os dados à coleção (como um insert em uma tabela)

In [35]:
collection.upsert(
    ids=ids,
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas
)
# nomes das chaves e conteúdos não podem ser diferentes, o chromadb espera como está

## 2. **Recuperação**

Quando um usuário envia uma consulta, o primeiro passo é encontrar os "chunks" de texto mais relevantes em nossa base de conhecimento. Este processo de busca semântica tem duas etapas:

1. **Incorporar a Consulta do Usuário**: A consulta do usuário (uma string de texto) deve ser convertida em um vetor de embedding usando o mesmo modelo que usamos para os documentos (text-embedding-004). Crucialmente, aqui usamos `task_type="RETRIEVAL_QUERY"` para otimizar o vetor para a tarefa de busca.

2. **Buscar por Similaridade**: O vetor da consulta é então usado para pesquisar no nosso armazenamento de vetores. A "similaridade" é tipicamente medida usando a **Similaridade de Cosseno (Cosine Similarity)**. Esta métrica calcula o cosseno do ângulo entre dois vetores. Um valor de `1` significa que os vetores apontam na mesma direção (semanticamente idênticos), `0` significa que são ortogonais (não relacionados), e `-1` significa que são opostos. Matematicamente, é calculado como o produto escalar dos vetores dividido pelo produto de suas magnitudes:
$$
S_C(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
$$

In [36]:
models_names[1]

'models/text-embedding-004'

In [ ]:
from typing import List
def recupera_chunks_relevantes(query: str, n_results: int = 5) -> List[str]:
    query_embedding_result = genai.embed_content(
        model=models_names[1],
        content=query,
        task_type="RETRIEVAL_QUERY"
    )
    query_embedding = query_embedding_result['embedding']

    # Realiza a busca por similaridade na coleção
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    return results['documents']

In [ ]:
# Exemplo de uso:
user_query = "Aprovado"
relevant_chunks = recupera_chunks_relevantes(user_query)[0]
print(f"{len(relevant_chunks)} Chunk(s) relevante(s) encontrado(s):")
for i, chunk in enumerate(relevant_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

## 3. **Geração**

Nesta etapa vamos usar os "chunks" recuperados como contexto para um modelo generativo.

Esta é a etapa mais crítica para cumprir a restrição principal do usuário: garantir que o chatbot responda apenas com base nas informações fornecidas. Devemos construir um prompt que instrua o modelo Gemini a abandonar seu conhecimento geral e a se ater estritamente ao contexto que fornecemos. Isso é alcançado através de uma **engenharia de prompt**.

In [61]:
def criar_prompt_rag(query: str, retrieved_chunks: List[str]) -> str:

    # Persona/Instrução de Papel: Diga ao modelo qual é o seu papel
    papel = "Você é um assistente especialista. Sua tarefa é responder à pergunta do usuário com base exclusivamente no contexto fornecido."

    # Instrução de Tarefa Clara: Diga ao modelo para basear sua resposta apenas no contexto.
    terefa = "Não utilize nenhum conhecimento externo ou treinamento prévio."

    # Instrução de Fallback: Diga ao modelo o que fazer se a resposta não estiver no contexto
    # fallback = "Se a informação para responder à pergunta não estiver no contexto ou no histórico da conversa, você deve declarar:'Não consigo responder a esta pergunta com base nos documentos fornecidos.'"
    fallback = (
        "Se a informação para responder à pergunta não estiver no CONTEXTO dos documentos nem no HISTÓRICO RESUMIDO, responda exatamente: "
        "'Não consigo responder a esta pergunta com base nos documentos fornecidos.' "
        "Para perguntas sobre o próprio histórico da conversa (ex.: 'qual foi minha primeira pergunta?'), consulte o HISTÓRICO RESUMIDO ou o HISTÓRICO BRUTO se for mais preciso. "
        "Não use conhecimento externo além desses."
    )
    # Contexto: Os "chunks" recuperados.
    contexto = "\n\n---\n\n".join(retrieved_chunks)

    # Pergunta: A pergunta original do usuário.
    pergunta = query

    prompt = f'{papel}\n{terefa}\n{fallback}\nCONTEXTO:\n---{contexto}\n---\nPERGUNTA:\n{pergunta}\nRESPOSTA:\n'

    return prompt

In [62]:
print(criar_prompt_rag("Aprovado", relevant_chunks))

Você é um assistente especialista. Sua tarefa é responder à pergunta do usuário com base exclusivamente no contexto fornecido.
Não utilize nenhum conhecimento externo ou treinamento prévio.
Se a informação para responder à pergunta não estiver no CONTEXTO dos documentos nem no HISTÓRICO RESUMIDO, responda exatamente: 'Não consigo responder a esta pergunta com base nos documentos fornecidos.' Para perguntas sobre o próprio histórico da conversa (ex.: 'qual foi minha primeira pergunta?'), consulte o HISTÓRICO RESUMIDO ou o HISTÓRICO BRUTO se for mais preciso. Não use conhecimento externo além desses.
CONTEXTO:
---das prestações pagas em benefício do credor que, em razão 
do inadimplemento, pleitear a resolução do contrato e a 
retomada do produto alienado.
§ 1° (Vetado).
§ 2º Nos contratos do sistema de consórcio de 
produtos duráveis, a compensação ou a restituição das 
parcelas quitadas, na forma deste artigo, terá descontada, 
além da vantagem econômica auferida com a fruição, os 
pre

**Resposta Final**

In [63]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold # coleção de constantes (ENUM)

In [64]:
model_list = []
for i,  model in enumerate(genai.list_models()):
  if 'generateContent' in model.supported_generation_methods: #lista modelos generativos
    print(f'{i-1} - {model.name}')
    model_list.append(model.name)

0 - models/gemini-2.5-pro-preview-03-25
1 - models/gemini-2.5-flash-preview-05-20
2 - models/gemini-2.5-flash
3 - models/gemini-2.5-flash-lite-preview-06-17
4 - models/gemini-2.5-pro-preview-05-06
5 - models/gemini-2.5-pro-preview-06-05
6 - models/gemini-2.5-pro
7 - models/gemini-2.0-flash-exp
8 - models/gemini-2.0-flash
9 - models/gemini-2.0-flash-001
10 - models/gemini-2.0-flash-exp-image-generation
11 - models/gemini-2.0-flash-lite-001
12 - models/gemini-2.0-flash-lite
13 - models/gemini-2.0-flash-preview-image-generation
14 - models/gemini-2.0-flash-lite-preview-02-05
15 - models/gemini-2.0-flash-lite-preview
16 - models/gemini-2.0-pro-exp
17 - models/gemini-2.0-pro-exp-02-05
18 - models/gemini-exp-1206
19 - models/gemini-2.0-flash-thinking-exp-01-21
20 - models/gemini-2.0-flash-thinking-exp
21 - models/gemini-2.0-flash-thinking-exp-1219
22 - models/gemini-2.5-flash-preview-tts
23 - models/gemini-2.5-pro-preview-tts
24 - models/learnlm-2.0-flash-experimental
25 - models/gemma-3-1b-

In [ ]:
generation_config = {
    "candidate_count": 1,
    "temperature": 1
}
safety_settings = { 
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [ ]:
# model = genai.GenerativeModel(
#     model_name=model_list[8],
#     generation_config=generation_config,
#     safety_settings=safety_settings # opcional, mas recomendado
#     )
model = genai.GenerativeModel(
    model_name=model_list[8],
    generation_config=generation_config
    )

In [ ]:
query = input("Prompt do usuário: ")
print("Usuário: ", query)
query_chunks = recupera_chunks_relevantes(query)[0]
prompt_RAG = criar_prompt_rag(query, query_chunks)
response = model.generate_content(prompt_RAG)
print(f'Modelo: {response.text}')

O horário de funcionamento é de segunda a sexta das 7h às 19h e sábado das 7h às 12h. Domingos e feriados a clínica está fechada.


**Chat continuo**

In [ ]:
chat = model.start_chat(history=[])

query = ""
while query != "fim":
  query = input()
  print("Usuário: ", query)
  query_chunks = recupera_chunks_relevantes(query)[0]
  prompt_RAG = criar_prompt_rag(query, query_chunks)
  response = chat.send_message(prompt_RAG)
  print(f'Chatbot: {response.text}')
  